In [ ]:
//*[@id="content"]/div[2]/div[2]/div/p[2]
//*[@id="content"]/div[2]/div[2]/div/p[2]
//*[@id="content"]/div[2]/div[2]/div/p[2]
//*[@id="content"]/div[2]/div[2]/div/p[2]

//*[@id="content"]/div[2]/div[3]/div/div[2]/div/div/div[2]/div/div[2]/div

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://eungdapso.seoul.go.kr/exp/pub/complaint_pub_lis.do"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/115.0.0.0 Safari/537.36",
    "Content-Type": "application/x-www-form-urlencoded"
}

payload = {
    "cp": 1,
    "sd": "2024-07-18",
    "ed": "2025-07-18",
    "sk": "",
    "sv": ""
}

res = requests.post(url, headers=headers, data=payload)
res.raise_for_status()

soup = BeautifulSoup(res.text, "html.parser")
print(soup.prettify()[:1000])  # 응답 확인용 일부만 출력


<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta charset="utf-8"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=yes" name="viewport"/>
  <link href="/images/logo_icon_114.png" rel="shortcut icon"/>
  <link href="/images/logo_icon_114.png" rel="apple-touch-icon"/>
  <link href="/css/reset.css" rel="stylesheet"/>
  <link href="/css/layout.css?v=20250401" rel="stylesheet"/>
  <link href="/css/swiper.min.css" rel="stylesheet"/>
  <link href="/css/jqueryui.css" rel="stylesheet"/>
  <link href="/css/contents.css?v=20250429" rel="stylesheet"/>
  <script src="/js/jquery-1.12.4.min.js">
  </script>
  <script src="/js/jquery.cookie.js">
  </script>
  <script src="/js/hoverIntent.js">
  </script>
  <script src="/js/swiper.min.js">
  </script>
  <script src="/js/iscroll.js">
  </script>
  <script src="/js/front.js?v=20250401">
  </script>
  <scri

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# 정확한 ChromeDriver 경로 입력
CHROMEDRIVER_PATH = r"C:\Aicamp\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# (선택) 크롬 바이너리 경로 지정 (Chrome for Testing 경로일 경우)
# CHROME_BINARY_PATH = r"C:\Program Files\Google\Chrome\Application\chrome.exe"  # 또는 chrome-for-testing 위치

options = Options()
# options.binary_location = CHROME_BINARY_PATH  # 필요 시 주석 해제
options.add_argument("--headless")
options.add_argument("--disable-gpu")

service = Service(executable_path=CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.google.com")
print("✅ ChromeDriver 연결 성공!")

driver.quit()


✅ ChromeDriver 연결 성공!


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# 경로 설정
CHROMEDRIVER_PATH = r"C:\Aicamp\chromedriver-win64\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")

service = Service(executable_path=CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

list_url = "https://eungdapso.seoul.go.kr/exp/pub/complaint_pub_lis.do"
driver.get(list_url)

data = []
uid = 1
MAX_PAGE = 28  # 필요에 따라 조정

for page in range(1, MAX_PAGE + 1):
    print(f"📄 페이지 {page} 수집 중...")
    driver.get(f"{list_url}?cp={page}")
    time.sleep(1)

    links = driver.find_elements(By.CSS_SELECTOR, "table tbody tr td a")
    hrefs = [link.get_attribute("href") for link in links]

    for href in hrefs:
        try:
            driver.get(href)
            time.sleep(0.5)
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # 민원요지
            req = soup.select_one(
                "#content > div.subcont_contwrap.skintype2 > div.board_viewer_contbox.sclinkage_cont > div > p:nth-child(2)"
            )
            req_text = req.get_text(strip=True) if req else ""

            # 민원답변 내용
            resp = soup.select_one(
                "#content > div.subcont_contwrap.skintype2 > div.indent_vitem_w > div > div.indent_vcontlow > div > div > div:nth-child(2) > div > div.res_hcell.restd > div"
            )
            resp_text = resp.get_text(separator=" ", strip=True) if resp else ""

            if req_text and resp_text:
                data.append({
                    "id": uid,
                    "질문": req_text,
                    "본문": resp_text
                })
                uid += 1

        except Exception as e:
            print(f"❌ 오류 발생 (민원 상세): {e}")
            continue

driver.quit()

df = pd.DataFrame(data)
df.to_csv("민원_크롤링_결과.csv", index=False, encoding="utf-8-sig")
print(f"✅ 완료: 총 {len(df)}건 저장됨.")


📄 페이지 1 수집 중...
❌ 오류 발생 (민원 상세): Message: unknown error: unsupported protocol
  (Session info: chrome=138.0.7204.102)
Stacktrace:
	GetHandleVerifier [0x0x7ff66f26e925+77845]
	GetHandleVerifier [0x0x7ff66f26e980+77936]
	(No symbol) [0x0x7ff66f029cda]
	(No symbol) [0x0x7ff66f02db8d]
	(No symbol) [0x0x7ff66f0d18be]
	(No symbol) [0x0x7ff66f0a88ca]
	(No symbol) [0x0x7ff66f0d0b07]
	(No symbol) [0x0x7ff66f0a86a3]
	(No symbol) [0x0x7ff66f071791]
	(No symbol) [0x0x7ff66f072523]
	GetHandleVerifier [0x0x7ff66f54683d+3059501]
	GetHandleVerifier [0x0x7ff66f540bfd+3035885]
	GetHandleVerifier [0x0x7ff66f5603f0+3164896]
	GetHandleVerifier [0x0x7ff66f288c2e+185118]
	GetHandleVerifier [0x0x7ff66f29053f+216111]
	GetHandleVerifier [0x0x7ff66f2772d4+113092]
	GetHandleVerifier [0x0x7ff66f277489+113529]
	GetHandleVerifier [0x0x7ff66f25e288+10616]
	BaseThreadInitThunk [0x0x7ffb22f2e8d7+23]
	RtlUserThreadStart [0x0x7ffb23d1c34c+44]

❌ 오류 발생 (민원 상세): Message: unknown error: unsupported protocol
  (Session info:

KeyboardInterrupt: 

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

CHROMEDRIVER_PATH = r"C:\Aicamp\chromedriver-win64\chromedriver-win64\chromedriver.exe"

options = Options()
# ✅ headless 제거
# options.add_argument("--headless")
# ✅ 일반 브라우저처럼 보이게
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.102 Safari/537.36")

# ✅ 자동화 탐지 우회
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

service = Service(executable_path=CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://eungdapso.seoul.go.kr/exp/pub/complaint_pub_lis.do")
time.sleep(3)

print("✅ 민원 사이트 접속 성공")
driver.quit()


✅ 민원 사이트 접속 성공


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# ChromeDriver 경로
CHROMEDRIVER_PATH = r"C:\Aicamp\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# Selenium 옵션 설정 (headless 제거 + 탐지 우회)
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.7204.102 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# options.add_argument("--headless")  # ❌ 제거해야 사이트 접속 가능

# 드라이버 실행
service = Service(executable_path=CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# 리스트 페이지 URL
list_url = "https://eungdapso.seoul.go.kr/exp/pub/complaint_pub_lis.do"

data = []
uid = 1
MAX_PAGE = 28  # 페이지 수 조정 가능

for page in range(1, MAX_PAGE + 1):
    print(f"📄 페이지 {page} 수집 중...")
    driver.get(f"{list_url}?cp={page}")
    time.sleep(1.5)

    links = driver.find_elements(By.CSS_SELECTOR, "table tbody tr td a")
    hrefs = [link.get_attribute("href") for link in links]

    for href in hrefs:
        try:
            driver.get(href)
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # 민원요지
            req = soup.select_one(
                "#content > div.subcont_contwrap.skintype2 > div.board_viewer_contbox.sclinkage_cont > div > p:nth-child(2)"
            )
            req_text = req.get_text(strip=True) if req else ""

            # 민원답변 내용
            resp = soup.select_one(
                "#content > div.subcont_contwrap.skintype2 > div.indent_vitem_w > div > div.indent_vcontlow > div > div > div:nth-child(2) > div > div.res_hcell.restd > div"
            )
            resp_text = resp.get_text(separator=" ", strip=True) if resp else ""

            if req_text and resp_text:
                data.append({
                    "id": uid,
                    "질문": req_text,
                    "본문": resp_text
                })
                uid += 1
        except Exception as e:
            print(f"❌ 오류 발생 (민원 상세): {e}")
            continue

driver.quit()

# CSV 저장
df = pd.DataFrame(data)
df.to_csv("민원_크롤링_결과.csv", index=False, encoding="utf-8-sig")
print(f"✅ 완료: 총 {len(df)}건 저장됨.")


📄 페이지 1 수집 중...
❌ 오류 발생 (민원 상세): Message: unknown error: unsupported protocol
  (Session info: chrome=138.0.7204.102)
Stacktrace:
	GetHandleVerifier [0x0x7ff66f26e925+77845]
	GetHandleVerifier [0x0x7ff66f26e980+77936]
	(No symbol) [0x0x7ff66f029cda]
	(No symbol) [0x0x7ff66f02db8d]
	(No symbol) [0x0x7ff66f0d18be]
	(No symbol) [0x0x7ff66f0a88ca]
	(No symbol) [0x0x7ff66f0d0b07]
	(No symbol) [0x0x7ff66f0a86a3]
	(No symbol) [0x0x7ff66f071791]
	(No symbol) [0x0x7ff66f072523]
	GetHandleVerifier [0x0x7ff66f54683d+3059501]
	GetHandleVerifier [0x0x7ff66f540bfd+3035885]
	GetHandleVerifier [0x0x7ff66f5603f0+3164896]
	GetHandleVerifier [0x0x7ff66f288c2e+185118]
	GetHandleVerifier [0x0x7ff66f29053f+216111]
	GetHandleVerifier [0x0x7ff66f2772d4+113092]
	GetHandleVerifier [0x0x7ff66f277489+113529]
	GetHandleVerifier [0x0x7ff66f25e288+10616]
	BaseThreadInitThunk [0x0x7ffb22f2e8d7+23]
	RtlUserThreadStart [0x0x7ffb23d1c34c+44]

❌ 오류 발생 (민원 상세): Message: unknown error: unsupported protocol
  (Session info:

KeyboardInterrupt: 